In [24]:
!pip install --upgrade tableauserverclient


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [27]:
import tableauserverclient as TSC

def get_workbooks_paginated(server, auth, page_size=100):
    with server.auth.sign_in(auth):
        pagination_item = TSC.Pager(page_size)
        all_workbooks = []
        
        while True:
            workbooks, pagination_item = server.workbooks.get(req_options=pagination_item)
            print(f"Retrieved {len(workbooks)} workbooks")
            all_workbooks.extend(workbooks)
            
            if not pagination_item.has_next_page:
                break
        
        return all_workbooks

# Use this function after successful authentication
if check_connection(server, tableau_auth):
    print("\nAttempting to retrieve workbooks...")
    try:
        workbooks = get_workbooks_paginated(server, tableau_auth)
        print(f"Successfully retrieved {len(workbooks)} workbooks")
        # Process workbooks here
    except Exception as e:
        print(f"Error retrieving workbooks: {str(e)}")

Authentication successful!
Connected to Tableau Server version: 2023.1.13
Logged in as: vlad.parakhin@okta.com (ID: 417397d3-606f-464f-a941-4d7325b05574)
Number of accessible workbooks: 100
First 5 workbook names:
- App Dashboard
- Comm - Weekly Update
- Okta For Good Dashboard
- Account Growth
- Current Subscriptions by Account
Connection failed. Error: 

	405000: Method Not Allowed
		The HTTP method 'GET' is not supported for the given resource


In [28]:
def simple_api_call(server, auth):
    with server.auth.sign_in(auth):
        # Try to get server info
        server_info = server.server_info.get()
        print(f"Server info: {server_info.product_version}")
        
        # Try to get current user info
        current_user = server.users.get_by_id(server.user_id)
        print(f"Current user: {current_user.name}")

# Use this after authentication
if check_connection(server, tableau_auth):
    print("\nAttempting a simple API call...")
    try:
        simple_api_call(server, tableau_auth)
    except Exception as e:
        print(f"Error in simple API call: {str(e)}")

Authentication successful!
Connected to Tableau Server version: 2023.1.13
Logged in as: vlad.parakhin@okta.com (ID: 417397d3-606f-464f-a941-4d7325b05574)
Number of accessible workbooks: 100
First 5 workbook names:
- App Dashboard
- Comm - Weekly Update
- Okta For Good Dashboard
- Account Growth
- Current Subscriptions by Account
Connection failed. Error: 

	405000: Method Not Allowed
		The HTTP method 'GET' is not supported for the given resource


In [29]:
import tableauserverclient as TSC
from requests.exceptions import RequestException

def test_tableau_connection(server_url, token_name, personal_access_token, site_id):
    tableau_auth = TSC.PersonalAccessTokenAuth(
        token_name=token_name,
        personal_access_token=personal_access_token,
        site_id=site_id
    )
    server = TSC.Server(server_url, use_server_version=True)
    
    try:
        with server.auth.sign_in(tableau_auth):
            print("Authentication successful!")
            server_info = server.server_info.get()
            print(f"Connected to Tableau Server version: {server_info.product_version}")
    except TSC.ServerResponseError as e:
        print(f"Server response error: {e.code} - {e.detail}")
    except RequestException as e:
        print(f"Network error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Use the function
server_url = "https://tableau.atko.biz"
token_name = "collab"
personal_access_token = "DY3rtmyfSAuhKYGtod8EeQ==:g5OLVbcq5WebjMJRIOIxH8iHHmhYwPyJ"
site_id = ""  # Leave empty for default site

test_tableau_connection(server_url, token_name, personal_access_token, site_id)

Authentication successful!
Connected to Tableau Server version: 2023.1.13


In [31]:
import tableauserverclient as TSC
from requests.exceptions import RequestException
import pandas as pd

def extract_tableau_metadata(server_url, token_name, personal_access_token, site_id):
    tableau_auth = TSC.PersonalAccessTokenAuth(
        token_name=token_name,
        personal_access_token=personal_access_token,
        site_id=site_id
    )
    server = TSC.Server(server_url, use_server_version=True)
    
    results = []
    
    try:
        with server.auth.sign_in(tableau_auth):
            print("Authentication successful!")
            server_info = server.server_info.get()
            print(f"Connected to Tableau Server version: {server_info.product_version}")
            
            # Get all workbooks with pagination handling
            all_workbooks = []
            req_options = TSC.RequestOptions(pagenumber=1)
            
            while True:
                workbooks_page, pagination_item = server.workbooks.get(req_options)
                all_workbooks.extend(workbooks_page)
                
                if len(all_workbooks) < pagination_item.total_available:
                    print(f"Fetching more workbooks... {len(all_workbooks)} of {pagination_item.total_available}")
                    req_options.pagenumber += 1  # Move to the next page
                else:
                    break

            print(f"Total workbooks found: {len(all_workbooks)}")

            for workbook in all_workbooks:
                print(f"Processing workbook: {workbook.name}")
                
                # Try populating views
                try:
                    server.workbooks.populate_views(workbook)
                    print(f"Total views in workbook {workbook.name}: {len(workbook.views)}")
                except TSC.ServerResponseError as e:
                    print(f"Failed to populate views for workbook {workbook.name}: {e}")
                    continue
                
                # Try populating connections
                try:
                    server.workbooks.populate_connections(workbook)
                    print(f"Total connections in workbook {workbook.name}: {len(workbook.connections)}")
                except TSC.ServerResponseError as e:
                    print(f"Failed to populate connections for workbook {workbook.name}: {e}")
                    continue
                
                for view in workbook.views:
                    for connection in workbook.connections:
                        results.append({
                            "Workbook": workbook.name,
                            "View": view.name,
                            "Datasource": connection.datasource_name if connection.datasource_name else connection.connection_type
                        })
                        print(f"  View: {view.name}, Datasource: {connection.datasource_name if connection.datasource_name else connection.connection_type}")
            
            print(f"Total results collected: {len(results)}")
            
            if results:
                df = pd.DataFrame(results)
                csv_filename = 'tableau_metadata.csv'
                df.to_csv(csv_filename, index=False)
                print(f"Data has been saved to '{csv_filename}'.")
                return df
            else:
                print("No results were collected. Check your permissions and workbook accessibility.")
                return None
                
    except TSC.ServerResponseError as e:
        print(f"Server response error: {e.code} - {e.detail}")
    except RequestException as e:
        print(f"Network error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    return None

# Use the function
server_url = "https://tableau.atko.biz"
token_name = "collab"
personal_access_token = "DY3rtmyfSAuhKYGtod8EeQ==:g5OLVbcq5WebjMJRIOIxH8iHHmhYwPyJ"
site_id = ""  # Leave empty for default site

df = extract_tableau_metadata(server_url, token_name, personal_access_token, site_id)

if df is not None:
    print("\nFirst few rows of extracted data:")
    print(df.head())

Authentication successful!
Connected to Tableau Server version: 2023.1.13
Fetching more workbooks... 100 of 525
Fetching more workbooks... 200 of 525
Fetching more workbooks... 300 of 525
Fetching more workbooks... 400 of 525
Fetching more workbooks... 500 of 525
Total workbooks found: 525
Processing workbook: App Dashboard
Total views in workbook App Dashboard: 6
Total connections in workbook App Dashboard: 1
  View: Top Apps, Datasource: Customer App Usage - Current SNOW
  View: Customers Using Apps, Datasource: Customer App Usage - Current SNOW
  View: Customers Using 2 Apps, Datasource: Customer App Usage - Current SNOW
  View: Customers Using 3 Apps, Datasource: Customer App Usage - Current SNOW
  View: Customers Not Using App, Datasource: Customer App Usage - Current SNOW
  View: Other Apps Used, Datasource: Customer App Usage - Current SNOW
Processing workbook: Comm - Weekly Update
Total views in workbook Comm - Weekly Update: 5
Total connections in workbook Comm - Weekly Update